In [5]:
import scanpy as sc
import pandas as pd

In [223]:
Wang_h_adult_and_T2D = sc.read_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/human_diabetic_dataset_Wang2023/Wang_h_adult_and_T2D.h5ad")
mouse_atlas_healthy_adult = sc.read_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/mouse_pancreatic_atlas/mouse_atlas_healthy_adult.h5ad")
Elgmal_h_adult_and_diabetic = sc.read_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/extra_human_datasets/Elgamal/Elgmal_h_adult_and_diabetic.h5ad")

In [6]:
orth = pd.read_table("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/mouse_human_homology.tsv")

In [21]:
orth

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Human gene stable ID,Human gene name,Query protein or transcript ID,Human homology type
0,ENSMUSG00000064341,ENSMUSG00000064341.1,ENSMUST00000082392,ENSMUST00000082392.1,ENSG00000198888,MT-ND1,ENSMUSP00000080991,ortholog_one2one
1,ENSMUSG00000064345,ENSMUSG00000064345.1,ENSMUST00000082396,ENSMUST00000082396.1,ENSG00000198763,MT-ND2,ENSMUSP00000080992,ortholog_one2one
2,ENSMUSG00000064351,ENSMUSG00000064351.1,ENSMUST00000082402,ENSMUST00000082402.1,ENSG00000198804,MT-CO1,ENSMUSP00000080993,ortholog_one2one
3,ENSMUSG00000064354,ENSMUSG00000064354.1,ENSMUST00000082405,ENSMUST00000082405.1,ENSG00000198712,MT-CO2,ENSMUSP00000080994,ortholog_one2one
4,ENSMUSG00000064356,ENSMUSG00000064356.1,ENSMUST00000082407,ENSMUST00000082407.1,ENSG00000228253,MT-ATP8,ENSMUSP00000080995,ortholog_one2one
...,...,...,...,...,...,...,...,...
105184,ENSMUSG00000068115,ENSMUSG00000068115.14,ENSMUST00000128564,ENSMUST00000128564.8,ENSG00000101004,NINL,ENSMUSP00000105522,ortholog_one2one
105185,ENSMUSG00000068115,ENSMUSG00000068115.14,ENSMUST00000149979,ENSMUST00000149979.2,ENSG00000101004,NINL,ENSMUSP00000105522,ortholog_one2one
105186,ENSMUSG00000068115,ENSMUSG00000068115.14,ENSMUST00000128627,ENSMUST00000128627.2,ENSG00000101004,NINL,ENSMUSP00000105522,ortholog_one2one
105187,ENSMUSG00000053916,ENSMUSG00000053916.5,ENSMUST00000066640,ENSMUST00000066640.5,ENSG00000170191,NANP,ENSMUSP00000063895,ortholog_one2one


In [232]:
Wang_obs = Wang_h_adult_and_T2D.obs
Wang_obs["dataset"] = "Wang"
Wang_obs["organism"] = "Homo sapiens"
Wang_obs = Wang_obs[["disease", "dataset", "organism"]]

Wang_h_adult_and_T2D.obs = Wang_obs

Wang_var = Wang_h_adult_and_T2D.var
Wang_var = Wang_var.copy()
Wang_var["name"] = Wang_var.index

In [233]:
orth_deduplicated = orth.drop_duplicates(subset="Human gene name", keep='first')
merged_df = Wang_var.merge(right=orth_deduplicated, left_on="name", right_on="Human gene name", how="left")
merged_df.index = merged_df["name"]
merged_df.index = merged_df.index.rename("EID")

In [234]:
Wang_h_adult_and_T2D.var = merged_df
Wang_h_adult_and_T2D.var["Gene stable ID"]
Wang_h_adult_and_T2D_orth = Wang_h_adult_and_T2D[:,~Wang_h_adult_and_T2D.var['Gene stable ID'].isna()]

Wang_h_adult_and_T2D_orth.var.index = Wang_h_adult_and_T2D_orth.var["Gene stable ID"]

Wang_h_adult_and_T2D_orth.var.index = Wang_h_adult_and_T2D_orth.var.index.rename("EID")
Wang_h_adult_and_T2D_orth.var = Wang_h_adult_and_T2D_orth.var.copy()

Wang_h_adult_and_T2D_orth = Wang_h_adult_and_T2D_orth[:,Wang_h_adult_and_T2D_orth.var["Human homology type"] == "ortholog_one2one"]

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [237]:
Elgmal_h_adult_and_diabetic_obs["Diabetes Status"] = Elgmal_h_adult_and_diabetic_obs["Diabetes Status"].replace("ND", "healthy")
Elgmal_h_adult_and_diabetic_obs["disease"] = Elgmal_h_adult_and_diabetic_obs["Diabetes Status"]

In [238]:
Elgmal_h_adult_and_diabetic_obs = Elgmal_h_adult_and_diabetic.obs
Elgmal_h_adult_and_diabetic_obs["dataset"] = "Elgmal"
Elgmal_h_adult_and_diabetic_obs["organism"] = "Homo sapiens"
Elgmal_h_adult_and_diabetic_obs = Elgmal_h_adult_and_diabetic_obs[["disease", "dataset", "organism"]]

In [239]:
Elgamal_var = Elgmal_h_adult_and_diabetic.var
Elgamal_var = Elgamal_var.copy()
Elgamal_var["name"] = Elgamal_var.index
Elgamal_var.index = Elgamal_var.index.rename("EID")

In [240]:
# Check for duplicate keys in both dataframes
print(Elgamal_var["name"].duplicated().sum())  # In Wang_var
print(orth["Human gene name"].duplicated().sum())  # In orth
merged_df.index = merged_df["name"]

0
86226


In [243]:
orth_deduplicated.index = orth_deduplicated["Human gene name"]
orth_deduplicated.index = orth_deduplicated.index.rename("EID")

In [244]:
merged_df = Elgamal_var.merge(right=orth_deduplicated, left_on="name", right_on="Human gene name", how="left")
merged_df.index = merged_df["Gene stable ID"].rename("EID")

In [245]:
Elgmal_h_adult_and_diabetic.var = merged_df
#Elgmal_h_adult_and_diabetic.var["Transcript stable ID"]

In [246]:
Elgmal_h_adult_and_diabetic = Elgmal_h_adult_and_diabetic[:,~Elgmal_h_adult_and_diabetic.var['Gene stable ID'].isna()]

Elgmal_h_adult_and_diabetic.var = Elgmal_h_adult_and_diabetic.var.copy()
Elgmal_h_adult_and_diabetic = Elgmal_h_adult_and_diabetic[:,Elgmal_h_adult_and_diabetic.var["Human homology type"] == "ortholog_one2one"]

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/anndata.py:882: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  self._set_dim_df(value, "var")


In [247]:
Elgmal_h_adult_and_diabetic.obs = Elgmal_h_adult_and_diabetic_obs

In [248]:
mouse_atlas_healthy_adult_obs = mouse_atlas_healthy_adult.obs

In [249]:
mouse_atlas_healthy_adult_obs["disease"] = "healthy"

In [250]:
mouse_atlas_healthy_adult.obs = mouse_atlas_healthy_adult_obs[["disease", "dataset", "organism"]]

In [251]:
mouse_atlas_healthy_adult.X = mouse_atlas_healthy_adult.raw.X

Preprocessing of datasets

In [267]:
sc.pp.normalize_per_cell(mouse_atlas_healthy_adult)

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[key_n_counts] = counts_per_cell


In [269]:
sc.pp.log1p(mouse_atlas_healthy_adult)

In [273]:
sc.pp.normalize_per_cell(Wang_h_adult_and_T2D_orth)

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell


In [274]:
sc.pp.log1p(Wang_h_adult_and_T2D_orth)

In [275]:
adata_concat = sc.concat([mouse_atlas_healthy_adult, Elgmal_h_adult_and_diabetic, Wang_h_adult_and_T2D_orth], join="inner")

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/merge.py:942: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(


In [278]:
adata_concat.write_h5ad("train_adata_baseline.h5ad")

In [279]:
sc.pp.highly_variable_genes(adata_concat, n_top_genes=3000)

In [281]:
adata_concat_highly_variable = adata_concat[:,adata_concat.var.highly_variable]

In [282]:
adata_concat_highly_variable.write_h5ad("train_adata_baseline_top3000hvg.h5ad")

In [301]:
Wang_h_adult_and_T2D_orth_concat = Wang_h_adult_and_T2D_orth[:,Wang_h_adult_and_T2D_orth.var.index.isin(adata_concat.var.index)]

In [302]:
Wang_h_adult_and_T2D_orth_concat.var = pd.DataFrame(Wang_h_adult_and_T2D_orth_concat.var.index)

In [303]:
Wang_h_adult_and_T2D_orth_concat.var.index = Wang_h_adult_and_T2D_orth_concat.var["EID"]

In [304]:
Wang_h_adult_and_T2D_orth_concat.var = Wang_h_adult_and_T2D_orth_concat.var.drop("EID", axis=1)

In [309]:
delattr(Wang_h_adult_and_T2D_orth_concat, "uns")

In [313]:
Wang_h_adult_and_T2D_orth_concat.obs = Wang_h_adult_and_T2D_orth_concat.obs.drop("n_counts", axis=1)

In [317]:
Wang_h_adult_and_T2D_orth_concat.write_h5ad("../data_to_predict/Wang_to_predict.h5ad")

### Ground truth dataset

In [318]:
mouse_atlas = sc.read_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/mouse_pancreatic_atlas/mouse_atlas_hrovatin.h5ad")

#### dbdb

In [320]:
dbdb = mouse_atlas[mouse_atlas.obs.dataset == "db/db"]

In [323]:
dbdb.obs.disease

index
CTTCCTTGTACCCAGC-1-MUC13640-VSG    type 2 diabetes mellitus
GTGCTTCCATTGCCTC-1-MUC13631-VSG    type 2 diabetes mellitus
ATAGGCTCATGCAGCC-1-MUC13639-VSG    type 2 diabetes mellitus
TACCTGCAGGAAAGGT-1-MUC13640-VSG    type 2 diabetes mellitus
TCGGATAGTAACCCGC-1-MUC13641-VSG    type 2 diabetes mellitus
                                             ...           
CCCTTAGTCGGAATGG-1-MUC13640-VSG    type 2 diabetes mellitus
CCCTTAGAGACAACAT-1-MUC13642-VSG    type 2 diabetes mellitus
AAATGGATCACCTCGT-1-MUC13631-VSG    type 2 diabetes mellitus
GTAGAAAAGACGGTCA-1-MUC13632-VSG    type 2 diabetes mellitus
ATGAGGGCATTCCTCG-1-MUC13632-VSG    type 2 diabetes mellitus
Name: disease, Length: 69745, dtype: category
Categories (2, object): ['type 2 diabetes mellitus', 'normal']

In [326]:
replace_dict = {
    "normal": "healthy",
    "type 2 diabetes mellitus": "T2D"
}

dbdb.obs.disease = dbdb.obs.disease.replace(replace_dict)

In [328]:
dbdb.obs = dbdb.obs[["disease", "dataset", "organism"]]

In [331]:
dbdb.var = pd.DataFrame(index=dbdb.var.index)

In [333]:
delattr(dbdb, "obsm")

In [335]:
delattr(dbdb, "uns")

In [337]:
dbdb.write_h5ad("../ground_truth/dbdb_ground_truth.h5ad")

In [341]:
pd.DataFrame(adata_concat_highly_variable.var.index, columns = ["genes"]).to_csv("top3000_hvg_baseline_train_data.h5ad")

In [345]:
pd.DataFrame(index = adata_concat_highly_variable.var.index).to_csv("../top3000_hvg_baseline_train_data.h5ad")

In [347]:
dbd_top3000_baseline_hvg = dbdb[:,dbdb.var.index.isin(adata_concat_highly_variable.var.index)]

In [349]:
dbd_top3000_baseline_hvg.write_h5ad("../ground_truth/dbdb_ground_truth_top3000_baseline_hvg.h5ad")

#### mSTZ

In [350]:
mSTZ = mouse_atlas[mouse_atlas.obs.dataset == "mSTZ"]

In [351]:
mSTZ.obs.disease

index
CAGAGAGCAACGATGG-1-G2-STZ    type 2 diabetes mellitus
CCCTCCTCAGCGTCCA-1-G5-STZ    type 2 diabetes mellitus
CCTACCATCCGAATGT-1-G6-STZ    type 2 diabetes mellitus
AGCGGTCCACTAGTAC-1-G3-STZ    type 2 diabetes mellitus
CTCGGGAAGGCAGTCA-1-G4-STZ    type 2 diabetes mellitus
                                       ...           
CCTATTAAGGTGGGTT-1-G8-STZ    type 2 diabetes mellitus
GGGATGAGTAACGACG-1-G4-STZ    type 2 diabetes mellitus
CGCTTCAAGATTACCC-1-G6-STZ    type 2 diabetes mellitus
CGTGTAATCGCGATCG-1-G4-STZ    type 2 diabetes mellitus
TAAACCGAGACGACGT-1-G8-STZ    type 2 diabetes mellitus
Name: disease, Length: 49545, dtype: category
Categories (2, object): ['type 2 diabetes mellitus', 'normal']

In [352]:
replace_dict = {
    "normal": "healthy",
    "type 2 diabetes mellitus": "T2D"
}

mSTZ.obs.disease = mSTZ.obs.disease.replace(replace_dict)

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/pandas/core/generic.py:5931: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self[name] = value


In [353]:
mSTZ.obs = mSTZ.obs[["disease", "dataset", "organism"]]

In [354]:
mSTZ.var = pd.DataFrame(index=mSTZ.var.index)

In [357]:
delattr(mSTZ, "obsm")

In [358]:
delattr(mSTZ, "uns")

In [359]:
mSTZ.write_h5ad("../ground_truth/mSTZ_ground_truth.h5ad")

In [362]:
mSTZ_top3000_baseline_hvg = mSTZ[:,mSTZ.var.index.isin(adata_concat_highly_variable.var.index)]

In [363]:
mSTZ_top3000_baseline_hvg.write_h5ad("../ground_truth/mSTZ_ground_truth_top3000_baseline_hvg.h5ad")

### Wnag to predict

In [372]:
Wang_h_adult_and_T2D_orth_concat[:,Wang_h_adult_and_T2D_orth_concat.var.index.isin(adata_concat_highly_variable.var.index)].write_h5ad("../data_to_predict/Wang_to_predict_top3000_baseline_hvg.h5ad")